## Import the data

This workbook relies on importing data as follows:

```
./epmv -v submit $(cat <<EOF
sample/ppr-batch/1864/629337.tgz
sample/ppr-batch/1854/625172.tgz
sample/ppr-batch/1879/680181.tgz
sample/ppr-batch/1859/627922.tgz
sample/ppr-batch/1899/696127.tgz
sample/ppr-batch/1869/633144.tgz
sample/ppr-batch/1874/676007.tgz
sample/ppr-batch/1884/685016.tgz
sample/ppr-batch/1889/692544.tgz
sample/ppr-batch/1904/802954.tgz
sample/ppr-batch/1894/693147.tgz
sample/ppr-batch/1909/804285.tgz
EOF
)
```

In [1]:
# import the query api module
import epmt_query as eq

## Basic Queries

The API has a only a few queries: `get_jobs`, `get_procs` and `get_thread_metrics`

Each of these operate at distinct levels: job, process and threads.

### Job Query

In [2]:
# let's get jobs, we use the job tag to select the jobs
jobs = eq.get_jobs(tag='exp_name:ESM4_historical_D151;exp_component:ocean_month_rho2_1x1deg',fmt='terse')
jobs

['629337',
 '625172',
 '680181',
 '627922',
 '696127',
 '633144',
 '676007',
 '685016',
 '692544',
 '802954',
 '693147',
 '804285']

In [35]:
# above we got a list of job ids. sometimes we want to see more details
# than just the job id. We can use `get_jobs` to convert between formats
jobs_df = eq.get_jobs(jobs, fmt='pandas')
display(jobs_df.columns.values)
jobs_df

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

,PERF_COUNT_SW_CPU_CLOCK,account,all_proc_tags,cancelled_write_bytes,cpu_time,created_at,delayacct_blkio_time,duration,end,env_changes_dict,...,time_oncpu,time_waiting,timeslices,updated_at,user,user+system,usertime,vol_ctxsw,wchar,write_bytes
0,866184046169,None,"[{'op_sequence': '139', 'op_instance': '3', 'o...",2448543744,959968389,2019-06-26 16:01:00.691579,0,12630660818,2019-06-09 22:23:53.234877,{},...,967130737639,48105200287,3175850,2019-06-26 16:01:00.691582,Jeffrey.Durachta,959968389,769390290,3132299,138638409056,134643470336
1,649725614194,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",3044032512,679701225,2019-06-26 16:36:44.851065,0,6532173945,2019-06-10 08:12:06.562689,{},...,681730630413,13802065001,824685,2019-06-26 16:36:44.851070,Jeffrey.Durachta,679701225,428874880,809138,74236894004,72541306880
2,576733745205,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",1998835712,623964730,2019-06-26 17:11:27.526872,0,6696039124,2019-06-10 11:50:58.082917,{},...,625961524504,19476877777,805888,2019-06-26 17:11:27.526876,Jeffrey.Durachta,623964730,478902294,792701,74236893737,70252703744
3,582075035986,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",1385582592,621768978,2019-06-26 17:47:26.478671,0,6625637678,2019-06-10 18:39:32.439890,{},...,623723072793,24198886180,826722,2019-06-26 17:47:26.478675,Jeffrey.Durachta,621768978,467152088,793749,74236867345,70780125184
4,605349533801,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",1465577472,640906121,2019-06-26 18:22:14.726809,0,10080732883,2019-06-14 11:18:38.154111,{},...,642678934515,23712986693,849400,2019-06-26 18:22:14.726813,Jeffrey.Durachta,640906121,450158690,832964,74236894547,73864601600
5,522980958427,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",1998835712,571561896,2019-06-26 18:53:58.558702,0,6009933600,2019-06-14 18:14:24.986076,{},...,573565404518,32443024755,818718,2019-06-26 18:53:58.558707,Jeffrey.Durachta,571561896,434850361,793333,74236809886,70259195904
6,403054040862,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",3392450560,427082965,2019-06-27 04:39:55.070758,0,7005618511,2019-06-15 09:49:24.210549,{},...,429280375226,11304090572,812300,2019-06-27 04:39:55.070762,Jeffrey.Durachta,427082965,332821891,799028,74236867987,70246367232
7,557685977561,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",3392417792,593701277,2019-06-27 05:12:35.879256,0,709300857,2019-06-16 14:06:18.129747,{},...,595771270622,18759054582,797027,2019-06-27 05:12:35.879259,Jeffrey.Durachta,593701277,457078582,783079,74236883941,70606073856
8,553117186630,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",1998848000,594222175,2019-06-27 05:46:19.237698,0,3340305357,2019-06-16 17:16:11.907347,{},...,596382718874,21984544439,801396,2019-06-27 05:46:19.237701,Jeffrey.Durachta,594222175,452663282,783373,74236938446,70251761664
9,574686010894,None,"[{'op_sequence': '56', 'op_instance': '5', 'op...",2347225088,607235263,2019-06-27 06:18:25.742892,0,3676905115,2019-06-17 07:22:16.747572,{},...,609161751217,17253128153,813225,2019-06-27 06:18:25.742895,Jeffrey.Durachta,607235263,468679853,797766,74236837177,70476115968


In [4]:
# if you prefer dealing with python lists and dictionaries,
# you can set fmt='dict'. Here we get a list of dictionaries
eq.get_jobs(jobs = jobs, fmt='dict')

[{'PERF_COUNT_SW_CPU_CLOCK': 866184046169,
  'account': None,
  'all_proc_tags': [{'op': 'untar', 'op_instance': '3', 'op_sequence': '139'},
   {'op': 'hsmput', 'op_instance': '1', 'op_sequence': '118'},
   {'op': 'untar', 'op_instance': '2', 'op_sequence': '130'},
   {'op': 'dmput', 'op_instance': '2', 'op_sequence': '190'},
   {'op': 'ncatted', 'op_instance': '1', 'op_sequence': '116'},
   {'op': 'untar', 'op_instance': '6', 'op_sequence': '166'},
   {'op': 'hsmget', 'op_instance': '13', 'op_sequence': '109'},
   {'op': 'fregrid', 'op_instance': '4', 'op_sequence': '150'},
   {'op': 'ncrcat', 'op_instance': '4', 'op_sequence': '136'},
   {'op': 'mv', 'op_instance': '13', 'op_sequence': '170'},
   {'op': 'cp', 'op_instance': '9', 'op_sequence': '158'},
   {'op': 'hsmget', 'op_instance': '7', 'op_sequence': '25'},
   {'op': 'ncatted', 'op_instance': '12', 'op_sequence': '174'},
   {'op': 'hsmget', 'op_instance': '10', 'op_sequence': '30'},
   {'op': 'hsmget', 'op_instance': '13', 'op_s

In [5]:
# there is a very useful format called ORM, this optimizes queries
# and lets you get the underlying Job (or Process) object directly
jobs_orm = eq.get_jobs(jobs, fmt='orm')
jobs_orm.count(), type(jobs_orm)

(12, pony.orm.core.Query)

The ORM format is powerful as it minimizes the number of SQL queries.
Let's see this in action. To do so, we need to enable SQL debug. This
first requires setting logging to INFO level or higher. Now, we will run a 
query first using a format other than ORM, say 'terse', and then using 
the 'orm' format. You will notice that in ORM format SQL queries are
"lazy-evaluated", leading to fewer queries.

It's only for the ORM type of result that you can use functions like:
`sum`, `count`, `avg`, etc. For other objects such as a list or pandas
dataframe, you would use functions like `len`.

In [6]:
eq.set_logging(1)
eq.set_sql_debug(True)

In [7]:
jobs = eq.get_jobs(tag='exp_component:ocean_month_rho2_1x1deg',fmt='terse')

INFO:pony.orm.sql:SELECT "j"."start", "j"."end", "j"."duration", "j"."proc_sums", "j"."created_at", "j"."updated_at", "j"."info_dict", "j"."env_dict", "j"."env_changes_dict", "j"."submit", "j"."jobid", "j"."jobname", "j"."jobscriptname", "j"."sessionid", "j"."exitcode", "j"."user", "j"."tags", "j"."account", "j"."queue", "j"."cpu_time"
FROM "job" "j"
WHERE ("j"."tags" #>> %(p1)s) = %(p2)s
INFO:pony.orm:RELEASE CONNECTION


In [8]:
jobs_orm =  eq.get_jobs(tag='exp_component:ocean_month_rho2_1x1deg',fmt='orm')

In [9]:
# Notice for the ORM, the query hasn't been run yet. Now, let's do a count
# of the jobs. You will see that rather than loading the jobs from the DB,
# only a COUNT sql query is run
jobs_orm.count()

INFO:pony.orm:GET CONNECTION FROM THE LOCAL POOL
INFO:pony.orm:SWITCH TO AUTOCOMMIT MODE
INFO:pony.orm.sql:SELECT COUNT(*)
FROM "job" "j"
WHERE ("j"."tags" #>> %(p1)s) = %(p2)s


12

In [10]:
# now let's remove the logging and sql debug to avoid cluttering the output
eq.set_sql_debug(False)
eq.set_logging(0)

In [26]:
# now let's see the job tags for each of the jobs in the ORM `Query` object
eq.select((j.jobid, j.tags) for j in jobs_orm)[:]

[('629337', {'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_18640101', 'exp_component': 'ocean_month_rho2_1x1deg', 'exp_time': '18640101', 'exp_name': 'ESM4_historical_D151', 'atm_res': 'c96l49', 'ocn_res': '0.5l75'}), ('625172', {'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_18540101', 'exp_component': 'ocean_month_rho2_1x1deg', 'exp_time': '18540101', 'exp_name': 'ESM4_historical_D151', 'atm_res': 'c96l49', 'ocn_res': '0.5l75'}), ('680181', {'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_18790101', 'exp_component': 'ocean_month_rho2_1x1deg', 'exp_time': '18790101', 'exp_name': 'ESM4_historical_D151', 'atm_res': 'c96l49', 'ocn_res': '0.5l75'}), ('627922', {'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1deg_18590101', 'exp_component': 'ocean_month_rho2_1x1deg', 'exp_time': '18590101', 'exp_name': 'ESM4_historical_D151', 'atm_res': 'c96l49', 'ocn_res': '0.5l75'}), ('696127', {'script_name': 'ESM4_historical_D151_ocean_month_rho2_1x1de

In [29]:
# some other useful queries might be for instance to order the jobs
# by duration
eq.get_jobs(jobs_orm, order='desc(j.duration)',fmt="pandas")[['jobid', 'tags', 'duration', 'exitcode']]

,jobid,tags,duration,exitcode
0,625172,{'script_name': 'ESM4_historical_D151_ocean_mo...,12630660818,0
1,676007,{'script_name': 'ESM4_historical_D151_ocean_mo...,10080732883,0
2,685016,{'script_name': 'ESM4_historical_D151_ocean_mo...,7005618511,0
3,629337,{'script_name': 'ESM4_historical_D151_ocean_mo...,6696039124,0
4,633144,{'script_name': 'ESM4_historical_D151_ocean_mo...,6625637678,0
5,627922,{'script_name': 'ESM4_historical_D151_ocean_mo...,6532173945,0
6,680181,{'script_name': 'ESM4_historical_D151_ocean_mo...,6009933600,0
7,802954,{'script_name': 'ESM4_historical_D151_ocean_mo...,3879024457,0
8,696127,{'script_name': 'ESM4_historical_D151_ocean_mo...,3676905115,0
9,693147,{'script_name': 'ESM4_historical_D151_ocean_mo...,3340305357,0


In [34]:
eq.get_jobs(jobs_orm, fltr='j.exitcode != 0', fmt='terse')

[]

As you get more familiar with the different job formats - `orm`, `dict`, `pandas`, `terse` -
it's worth pointing out some nuances. The `dict`, `terse` and `pandas` formats are
derived internally from the `orm` format. While there is generally a one-to-one
translation between the `orm` object attributes and the `dict` or `pandas` dataframe
columns, there is one attribute `proc_sums` that is initially in the ORM, and
is it's actually a dictionary. When converting from `orm` to the other formats,
the underlying key/value pairs of the dictionary are made available as top-level
fields of the `dict` or `pandas` dataframe. `proc_sums` represents aggregates across
the processes of a job:

In [37]:
j = jobs_orm.first()
j.proc_sums.keys()

dict_keys(['PERF_COUNT_SW_CPU_CLOCK', 'processor', 'rssmax', 'inblock', 'write_bytes', 'timeslices', 'majflt', 'guest_time', 'systemtime', 'all_proc_tags', 'cancelled_write_bytes', 'user+system', 'rchar', 'num_threads', 'time_waiting', 'invol_ctxsw', 'syscw', 'syscr', 'delayacct_blkio_time', 'usertime', 'outblock', 'num_procs', 'vol_ctxsw', 'minflt', 'wchar', 'read_bytes', 'time_oncpu', 'rdtsc_duration'])

Now, the fields shown above become available in other formats as top-level fields, while the `proc_sums`
field itself is masked.

In [39]:
j_df = eq.get_jobs(j, fmt='pandas')
j_df.columns.values

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

### Process Query

In [12]:
# If you want to get the processes belonging to a job
# here each row in the pandas dataframe contains one job process
# again, you can use the 'terse' fmt option to get just the list of database ids of the processes
eq.get_procs(['629337'], fmt='pandas')

,PERF_COUNT_SW_CPU_CLOCK,args,cancelled_write_bytes,created_at,delayacct_blkio_time,duration,end,exclusive_cpu_time,exename,exitcode,...,time_oncpu,time_waiting,timeslices,updated_at,user,user+system,usertime,vol_ctxsw,wchar,write_bytes
0,288875,^fre/.+,0,2019-06-26 17:11:35.152791,0,5014,2019-06-10 15:50:57.953311,4998,grep,0,...,5971828,4517378,6,2019-06-26 17:11:35.152794,Jeffrey.Durachta,4998,2999,5,0,0
1,77395,: n,0,2019-06-26 17:11:35.150577,0,82,2019-06-10 15:50:57.952353,3998,tr,0,...,4687213,8550701,6,2019-06-26 17:11:35.150580,Jeffrey.Durachta,3998,1999,4,0,0
2,105049,-c echo torque/6.0.2:moab/9.0.2:slurm/18.08:gl...,0,2019-06-26 17:11:35.148391,0,109,2019-06-10 15:50:57.943236,999,bash,0,...,1110693,4971699,1,2019-06-26 17:11:35.148394,Jeffrey.Durachta,999,999,0,203,0
3,1270150,-c echo torque/6.0.2:moab/9.0.2:slurm/18.08:gl...,0,2019-06-26 17:11:35.155110,0,18487,2019-06-10 15:50:57.954098,6998,bash,0,...,7958715,56548,10,2019-06-26 17:11:35.155112,Jeffrey.Durachta,6998,3999,8,0,0
4,102192,fredb,0,2019-06-26 17:11:35.146150,0,321,2019-06-10 15:50:57.923063,6998,which,0,...,7385108,88913,9,2019-06-26 17:11:35.146153,Jeffrey.Durachta,6998,3999,6,0,0
5,299989,-Gn,0,2019-06-26 17:11:35.143958,0,489,2019-06-10 15:50:57.903606,6998,id,0,...,7249342,101952,9,2019-06-26 17:11:35.143961,Jeffrey.Durachta,6998,3999,6,0,0
6,393361,-Gn,0,2019-06-26 17:11:35.141777,0,698,2019-06-10 15:50:57.892236,6998,id,0,...,7739709,152639,9,2019-06-26 17:11:35.141779,Jeffrey.Durachta,6998,2999,6,0,0
7,319789,-Gn,0,2019-06-26 17:11:35.139584,0,618,2019-06-10 15:50:57.868106,6998,id,0,...,7273913,96956,9,2019-06-26 17:11:35.139587,Jeffrey.Durachta,6998,2999,6,0,0
8,357973,-Gn,0,2019-06-26 17:11:35.137405,0,546,2019-06-10 15:50:57.856721,6998,id,0,...,7666008,165383,9,2019-06-26 17:11:35.137408,Jeffrey.Durachta,6998,3999,6,0,0
9,321774,-Gn,0,2019-06-26 17:11:35.135218,0,581,2019-06-10 15:50:57.832829,6998,id,0,...,7165056,107410,9,2019-06-26 17:11:35.135221,Jeffrey.Durachta,6998,3999,6,0,0


You could also pass in more than one job, in which case the returned processes
would be a superset of those across the jobs list:

In [13]:
procs = eq.get_procs(['629337', '625172'], fmt='orm')
procs.count()

15943

In [14]:
# suppose you want to figure out the unique set of operations
# across all the jobs of interest. We would pass in our list of
# jobs
eq.job_proc_tags(jobs_orm)

[{'op': 'rm', 'op_instance': '7', 'op_sequence': '146'},
 {'op': 'fregrid', 'op_instance': '2', 'op_sequence': '31'},
 {'op': 'hsmget', 'op_instance': '13', 'op_sequence': '86'},
 {'op': 'hsmget', 'op_instance': '10', 'op_sequence': '37'},
 {'op': 'hsmget', 'op_instance': '13', 'op_sequence': '80'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '24'},
 {'op': 'hsmput', 'op_instance': '1', 'op_sequence': '118'},
 {'op': 'timavg', 'op_instance': '9', 'op_sequence': '64'},
 {'op': 'rm', 'op_instance': '16', 'op_sequence': '75'},
 {'op': 'mv', 'op_instance': '13', 'op_sequence': '69'},
 {'op': 'mv', 'op_instance': '4', 'op_sequence': '143'},
 {'op': 'mv', 'op_instance': '18', 'op_sequence': '187'},
 {'op': 'hsmget', 'op_instance': '10', 'op_sequence': '48'},
 {'op': 'untar', 'op_instance': '2', 'op_sequence': '130'},
 {'op': 'ncatted', 'op_instance': '15', 'op_sequence': '82'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '21'},
 {'op': 'ncrcat', 'op_instance': '6', 'op_sequ

In [15]:
# now let's say we care about a particular operation. 
# Let's find the processes in the operation, and
# sort them by the cpu_time, and then see the top offenders
ncatted_procs = eq.get_procs(jobs_orm, \
                             tag = {'op': 'ncatted'}, \
                             order='desc(p.exclusive_cpu_time)', \
                             limit=10, \
                             fmt='pandas')
ncatted_procs[['jobid', 'tags', 'exename', 'duration', 'exclusive_cpu_time']]

,jobid,tags,exename,duration,exclusive_cpu_time
0,680181,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1256,58990
1,680181,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncdump,1112,53991
2,629337,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1143,48992
3,693147,"{'op_sequence': '41', 'op_instance': '5', 'op'...",ncatted,1118,48992
4,629337,"{'op_sequence': '32', 'op_instance': '3', 'op'...",ncatted,1119,48991
5,627922,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1037,47992
6,696127,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1082,47992
7,633144,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1085,47991
8,692544,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1053,47991
9,693147,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1042,46991


We could have used a more precise tag, such as `{'op': 'ncatted', 'op_sequence': 85}`,
for more granular selection. And, maybe an exename, such as `ncatted`.

In [16]:
procs = eq.get_procs(jobs_orm, tag='op:ncatted;op_sequence:85', \
                     fltr='p.exename == "ncatted"', \
                     order='desc(p.duration)', \
                     fmt='pandas')
procs[['jobid', 'tags', 'exename', 'duration', 'exclusive_cpu_time', 'exitcode']]

,jobid,tags,exename,duration,exclusive_cpu_time,exitcode
0,680181,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1256,58990,0
1,629337,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1143,48992,0
2,633144,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1085,47991,0
3,696127,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1082,47992,0
4,692544,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1053,47991,0
5,693147,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1042,46991,0
6,627922,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,1037,47992,0
7,804285,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,588,22995,0
8,676007,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,569,23995,0
9,802954,"{'op_sequence': '85', 'op_instance': '15', 'op...",ncatted,536,21996,0


Note that the `pandas` (and the `dict`) formats, in addition to having process-level data in each row, also have fields that represent metrics aggregated across the underlying threads of the process, such, as
`rssmax`, `exclusive_cpu_time`, and `rchar`.

In [40]:
procs.columns.values

array(['PERF_COUNT_SW_CPU_CLOCK', 'args', 'cancelled_write_bytes',
       'created_at', 'delayacct_blkio_time', 'duration', 'end',
       'exclusive_cpu_time', 'exename', 'exitcode', 'gen', 'group',
       'guest_time', 'host', 'id', 'inblock', 'inclusive_cpu_time',
       'invol_ctxsw', 'job', 'jobid', 'majflt', 'minflt', 'numtids',
       'outblock', 'parent', 'path', 'pgid', 'pid', 'ppid', 'processor',
       'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax', 'sid', 'start',
       'syscr', 'syscw', 'systemtime', 'tags', 'time_oncpu',
       'time_waiting', 'timeslices', 'updated_at', 'user', 'user+system',
       'usertime', 'vol_ctxsw', 'wchar', 'write_bytes'], dtype=object)

### Thread Query

In [18]:
# How about getting the threads metrics for these two processes?
eq.get_thread_metrics([3619, 3622])

,tid,start,end,usertime,systemtime,rssmax,minflt,majflt,inblock,outblock,...,syscw,read_bytes,write_bytes,cancelled_write_bytes,time_oncpu,time_waiting,timeslices,rdtsc_duration,PERF_COUNT_SW_CPU_CLOCK,process_pk
0,10957,1560519082926830,1560519082957492,8768,4384,4800,606,0,6,0,...,2,3328,257,0,13153030,0,3,79518868,1259188,3619
0,11417,1560519096483459,1560519096592013,12742,3185,5928,651,0,6,0,...,0,3328,0,0,15928333,20750,4,281572694,3944835,3622


## Getting familiar with useful metrics and keys

`get_jobs` and `get_procs` take a `fltr` and `order` option that can
filter and sort the output based on schema columns. 

In [19]:
# below we filter those processes of the job that exceed a certain
# wallclock time, and then sort them by the exclusive cpu time (user+system)
# fltr can be a lamdba function or a string
eq.get_procs('18431', fltr = lambda p: p.duration > 100000, order = 'desc(p.exclusive_cpu_time)', fmt='pandas')

ObjectNotFound: Job['18431']

### Useful metrics and keys

Below are some of the most useful keys in no particular order:

#### Job Keys
 - duration: this is the wallclock time in microseconds
 - cpu_time: user+system time aggregated across all processes of the job
 - start:    start time in microseconds since epoch
 - end:      end time in microseconds since epoch
 - jobid:    database id for job (unique)
 - exitcode: return code from job
 - tags:     dict of key/value pairs
 - processes:list of processes belonging to job
 
 #### Process Keys
 - duration: this is the wallclock time in microseconds
 - exclusive_cpu_time: user+system time for process (aggregated across it's threads)
 - inclusive_cpu_time: user+system time for the process and *all its descendants*
 - start:    start time in microseconds since epoch
 - end:      end time in microseconds since epoch
 - tags:     dict of key/value pairs
 - threads_df: json serialized dataframe of process threads (ADVANCED)
 - threads_sums: key/value pairs consisting of sums of thread metrics (ADVANCED)
 - numtids:  number of threads
 - exename
 - args
 - pid
 - ppid
 - id:       database ID for process
 - exitcode
 - parent
 - children
 - ancestors
 - descendants
 
 #### Thread Keys
 - usertime
 - systemtime
 - user+system
 - rssmax
 - majflt
 - read_bytes
 - write_bytes

## Case Study I

Let's walk through a contrived case study to get more familiar with the API.
Along the way we will touch on some advanced topics such as the ORM. You will
also see easy ways to navigate the process tree using the ORM.

Consider the shell script below:
```
$ cat sample/query/18431-job.sh 

#!/bin/bash
export PAPIEX_TAGS="prog:dircrawl;phase:/usr"
find /usr > /dev/null 2>&1

export PAPIEX_TAGS="prog:find;phase:stat"
(find /etc -exec stat {} \; ; ls -l /) > /dev/null 2>&1
sleep 10
```

In [ ]:
# ordinarily we would first find the job and then probe downwards
# You can use tags or fltr arguments to find the job
# As we did not include job tags in this script, let's just find the job using
# its job id
job = eq.get_jobs(jobs = ['18431'])[0]
job

In [ ]:
# now get the processes that are part of this job, let's sort them by the inclusive time
# we need to pass in the job id to restrict the query to a particular job
# the inclusive_cpu_time sums all the cpu times of the process and its dependents
# in this case you can see that after the top-level 'bash', the 'find' with the
# -exec stat shows up
procs = eq.get_procs(['18431'], order = 'desc(p.inclusive_cpu_time)', fmt='pandas')
procs

In [ ]:
# now let's try and see if one process was responsible for spawning too many processes
eq.get_procs(['18431'], fltr = 'count(p.children) > 100')

In [ ]:
# now let's walk through the process tree. To make this easy, we use the 'orm' format
# let's sort the processes by exclusive cpu time
# You will get a sorted list of ORM objects, let's see the top 10
procs = eq.get_procs(['18431'], order = 'desc(p.exclusive_cpu_time)', fmt='orm')[:10]
procs

In [ ]:
# lets pick up the first
p = procs[0]
p

In [ ]:
p.exename

In [ ]:
p.exename, p.args, p.duration, len(p.children), p.numtids

In [ ]:
parent = p.parent
parent

In [ ]:
parent.exename, parent.args, parent.pid, len(parent.children), len(parent.descendants)

In [ ]:
# let's see p's thread sums
p.threads_sums

In [ ]:
# let's get the thread dataframes for p
eq.get_thread_metrics(p)

## Case Study II

This uses the script below:

```
$ cat test-proctree.sh 
#!/bin/bash

export PAPIEX_TAGS="phase:/usr;utility:find"
find /usr > /dev/null 2>&1

export PAPIEX_TAGS="phase:/etc;utility:find"
sleep 10
(find /etc -exec stat {} \; ; ls -l /) > /dev/null 2>&1
```

Additionally, we have set a job tag, like so:
```
$ export EPMT_JOB_TAGS="experiment:file-system-explore"

$ ./epmt -a run ./test-proctree.sh
```

In [ ]:
# first let's get the job
# when you use the orm format, you can either use [:] to get a list
# from the ORM object, and then pick an index
j = eq.get_jobs(tags = {'experiment':'file-system-explore'}, fmt='orm')[:][0]

In [ ]:
j.jobid, j.duration, j.tags

In [ ]:
# let's now get focus on the processes for phase:/usr
p1 = eq.get_procs(j, tags = {'phase': '/usr'}, fmt='pandas')
p1[['exename', 'args', 'exclusive_cpu_time', 'inclusive_cpu_time', 'duration', 'exitcode']]

In [ ]:
# so there was only the one find process in the phase
# Now, let's turn our attention to the other phase
# we will use the ORM format first, as it returns a Query object
# without converting everything to a list. The query
# object has a count() method for quickly getting a count of the 
# number of rows. If you did not use the 'orm' format, then
# the full dataset will be retrieved from the database
p2 = eq.get_procs(j, tags = {'phase': '/etc'}, fmt='orm')
p2.count()

In [ ]:
# So a total of 3615 processes in the second phase
# let's re-run the query above, but this time with some sorting by time taken by the process
# and it's descendants. Let's limit the output to the top 5 results
# and let's get a pandas dataframe
p2 = eq.get_procs(j, tags = {'phase': '/etc'}, order='desc(p.inclusive_cpu_time)', limit=5, fmt='pandas')
p2[['exename', 'args', 'exclusive_cpu_time', 'inclusive_cpu_time', 'duration']]

In [ ]:
# As you can spot, the find within the subshell shows up at rank 2. The other two
# bash processes have virtually no exclusive times, the find both has a large 
# exclusive time, and one of the top inclusive times.

# Now let's see if any processes failed in the job
failed_procs = eq.get_procs(j, fltr='p.exitcode > 0', fmt='pandas')
failed_procs[['exename', 'args', 'exitcode', 'tags']]

In [ ]:
# So the find was also the one and only process that failed in the job

## Case Study - Linux Kernel Compile

Start by importing the data for this experiment (import takes around half an hour on my laptop):
```
$ ./epmt -v submit sample/kernel/run_output/
```

Let's review the script:
```
$ cat sample/kernel/build-linux-kernel.sh 
#!/bin/bash -e

# you will need the following deps installed:
#  sudo apt-get install build-essential libncurses-dev bison flex libssl-dev libelf-dev coreutils

# EPMT_JOB_TAGS='model:linux-kernel;compiler:gcc' ./epmt -a -j kernel-build-$(date +%Y%m%d-%H%M%S) run sample/kernel/build-linux-kernel.sh
#

build_dir=$(tempfile -p epmt -s build)
echo "creating build directory: $build_dir"
rm -rf $build_dir; mkdir -p $build_dir && cd $build_dir

# download
PAPIEX_TAGS="operation:download;operation_count:1;instance:1" wget https://cdn.kernel.org/pub/linux/kernel/v5.x/linux-5.1.7.tar.xz
PAPIEX_TAGS="operation:extract;operation_count:2;instance:1" tar -xf linux-5.1.7.tar.xz
cd linux-5.1.7

# configure
cp -v /boot/config-$(uname -r) .config
PAPIEX_TAGS="operation:configure;operation_count:3;instance:1" make olddefconfig

# build
PAPIEX_TAGS="operation:build;operation_count:4;instance:1" make -j $(nproc)
```

The job has a tag set: `model:linux-kernel;compiler:gcc`

Each of the download, extract, configure and build operations are marked using `PAPIEX_TAGS`.

In [ ]:
# start by locating the job in the database using tags
# you can specify tags as a dict or a string
# use fmt='terse' as we just want to know the job id
j = eq.get_jobs(tags = 'model:linux-kernel;compiler:gcc', fmt='terse')
j

In [ ]:
# let's get all the tags associated with the processes of the jobs
# This is a *very slow query* as all the processes for the job are loaded
# and the tags are filtered to get the unique tags
# If you already know the tags of the operations you care about,
# then this step is not needed
# TODO: Create set of unique process tags set during job import
eq.get_all_tags_in_job('kernel-build-20190606-150222')

In [ ]:
# let's see the processes in the download phase
download = eq.get_procs('kernel-build-20190606-150222', tags = 'operation:download', fmt='pandas')
download[['exename', 'args', 'exitcode', 'duration']]

In [ ]:
# So, there was only the single program wget and the duration shows the wallclock time
# Now let's sudy the configre phase. We expect it to have many processes. Whenever the 
# number of processes is large, it is a good idea to use order_by and limit, particularly
# if the format is dict or pandas. The 'orm' and 'terse' formats are usually fast already.
configure = eq.get_procs('kernel-build-20190606-150222', tags = 'operation:configure', fmt='terse')
configure

In [ ]:
# As you can see, that a lot of processes. Let's use order and limit to get a better understanding
# So, we will re-run the query but this time, we will sort by inclusive_cpu_time and get the top 10 processes
configure = eq.get_procs('kernel-build-20190606-150222', tags = 'operation:configure', order = 'desc(p.inclusive_cpu_time)', limit = 10, fmt='pandas')
configure[['exename', 'args', 'pid', 'duration', 'inclusive_cpu_time', 'exclusive_cpu_time']]

In [ ]:
# ADVANCED TOPIC:
# The idea below is to get the user familiar with the power of ORM
# operations, so we can get feedback and ideas for new API calls
# 

# If you just want to know the total time of an operation, and you could
# use database queries on the ORM directly. We need to import the ORM
# functions. You get sum, min, max and avg
# The big advantage is the speedup in the query whenever you use the ORM
# as there is lazy loading and optimized queries using db primitives
# TODO: Should we make a query for aggregate operations on an attribute?
from pony.orm import *
c = eq.get_procs('kernel-build-20190606-150222', tags = 'operation:configure', fmt='orm')
select(p.exclusive_cpu_time for p in c).sum()

In [ ]:
# another trick that works to get the max time for an operation is
# to find the process with the max value for duration. This works if
# you have a top-level process that spawned the rest
# Notice we use order and limit
root_build_proc = eq.get_procs('kernel-build-20190606-150222', tags = 'operation:build', order='desc(p.duration)', limit=5, fmt='pandas')
root_build_proc[['exename', 'args', 'duration', 'inclusive_cpu_time', 'exitcode']]

In [ ]:
# Above, you notice the build operation's root process 'make' took
# 20k seconds or about 6 hours! The inclusive_cpu_time can also be used
# to sort by chronological order in a linear script

# Now let's see if any process failed in the build phase
# If you use 'orm' you get access to 'count', which is superfast as it
# uses sql to a count directly rather than load all the fields of the matching processes
eq.get_procs('kernel-build-20190606-150222', tags = 'operation:build', fltr='p.exitcode != 0', fmt='orm').count()

In [ ]:
# Advanced topic:
# The orm also gives an easy way to navigate the process hierarchy
# Let's use the ORM directly to walk through the job
j = eq.get_jobs('kernel-build-20190606-150222', fmt='orm').first()
j

In [ ]:
# Notice we have a Job object. The processes in the job
# are available as j.processes
j.duration, j.cpu_time, j.exitcode, j.tags

In [ ]:
# let's see the process that took the max cpu time
max_cpu_proc = j.processes.order_by('desc(p.exclusive_cpu_time)').limit(1)[0]
max_cpu_proc.exename, max_cpu_proc.pid, max_cpu_proc.exclusive_cpu_time, max_cpu_proc.duration

In [ ]:
# let's get details on the build operation
b = eq.get_procs(j, tags = 'operation:build', fmt='orm')
b

In [ ]:
# Above we get a Query object, we can iterate over it, convert
# it to a list or get a slice of it
sorted_by_cpu = b.order_by('desc(p.inclusive_cpu_time)').limit(5)
sorted_by_cpu

In [ ]:
# observe that we don't actually do any queries until we start using
# the result
top_cpu = sorted_by_cpu[0]
top_cpu

In [ ]:
top_cpu.exename, top_cpu.args, top_cpu.duration, top_cpu.exclusive_cpu_time

In [ ]:
# now we get access to the parent/children/ancestors/descendats of this process
max(top_cpu.descendants.exitcode)

In [ ]:
# so one or more descendant processes failed, let's find which ones
failed = top_cpu.descendants.filter('p.exitcode != 0')
failed.count()

In [ ]:
# we can convert a Query object to a pandas dataframe anytime
# TODO: should we create an API call to convert ORM Query objects to
# pandas/dict?
import pandas as pd
df = pd.DataFrame([p.to_dict() for p in failed])
df[['exename', 'args', 'start', 'end', 'pid', 'ppid', 'exitcode']]

In [ ]:
# In this example we query aggregate times for different operations
# across jobs. This example uses data from two different runs of kernel
# compiles:
eq.get_jobs(fmt='terse')

In [ ]:
# first we ask for the aggregate metrics for single job
# Here, we don't specify any tags. For single jobs, when
# we don't specify the operation/tags, they are queried from the job
eq.op_metrics(jobs='kernel-build-20190606-150222', fmt='pandas', sql_debug=True)

In [ ]:
# Now let's run the same query against all jobs. In this case, we need
# to provide a list of tags (or a single tag) for the operation
eq.op_metrics(tags=['operation:build', 'operation:configure'])[['job','tags', 'exclusive_cpu_time','num_procs', 'rssmax']]

In [ ]:
# let's look at a particular job and see the processes with largest page faults
# across all threads for only the build operation
df = eq.get_procs('kernel-build-20190606-150222', tags='operation:build', order='desc(p.threads_sums["majflt"])', limit=5, fmt='pandas')
df[['exename', 'args', 'majflt', 'exclusive_cpu_time']]

## More examples

Let's do some useful queries that might form part of your workflow.
Some of these queries will use the ORM, so be forewarned.


In [ ]:
# As you may know for outlier detection we can only compare jobs with the
# same exp_name and exp_component. Let's do a query to count the number of jobs
# for each exp_component:
# For this we will use advanced ORM methods
from pony.orm import *
from models import *
q = select((count(j), j.tags['exp_component']) for j in Job)
list(q[:])

In [ ]:
# the 'ice_1x1deg' component seems to have the most jobs,
# let's look at those jobs
ice_1x1_jobs = eq.get_jobs(tags='exp_component:ice_1x1deg', fmt='terse') 
ice_1x1_jobs

In [ ]:
# Before doing outlier detection, let's look at operations in these jobs
# we use fold=True, to compact the tags dicts
ice_1x1_tags = eq.get_unique_process_tags([u'633109', u'625142', u'627902', u'629314'], fold=True)
ice_1x1_tags

In [ ]:
# That's a lot of tags. Let's exclude 'op_instance' and 'op_sequence' to
# reduce the list. 
ice_1x1_tags_excl = eq.get_unique_process_tags([u'633109', u'625142', u'627902', u'629314'], exclude=['op_instance', 'op_sequence'], fold=True)
ice_1x1_tags_excl

In [ ]:
# To get aggregate metrics by tags, the tags must not be in a folded form. 
# Also, let's not remove any tahs
ice_1x1_tags = eq.get_unique_process_tags([u'633109', u'625142', u'627902', u'629314'], fold=False)
len(ice_1x1_tags)

In [ ]:
# Now let's get the metrics aggregated by tag
# each row represents the aggregated metrics by tag for a particular job
df = eq.op_metrics([u'633109', u'625142', u'627902', u'629314'], tags=ice_1x1_tags, fmt='pandas')
display(df.columns.values)
df[['job', 'tags', 'duration', 'exclusive_cpu_time', 'num_procs', 'rssmax', 'majflt']]

In [ ]:
# Notice the page faults happen on op_sequence=206, op_instace=2 for each of the jobs